In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of windmark.core.structs failed: Traceback (most recent call last):
  File "/home/grantham/.cache/pypoetry/virtualenvs/windmark-gThMBSDt-py3.11/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/grantham/.cache/pypoetry/virtualenvs/windmark-gThMBSDt-py3.11/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/home/grantham/.pyenv/versions/3.11.8/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/grantham/windmark/windmark/core/structs.py", line 76, in <module>
    class Hyperparameters(pydantic.BaseModel):
  File "/home/gr

In [1]:
print("hello world")

hello world
